# Library

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import joblib
from functools import partial
import os
from concurrent.futures import ThreadPoolExecutor
from joblib import Parallel, delayed
from tqdm import tqdm
import itertools

from concurrent.futures import ProcessPoolExecutor
from astropy.stats import sigma_clip

from scipy.signal import savgol_filter
from scipy.optimize import minimize
from scipy import optimize

from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso



from sklearn.svm import SVR

from xgboost import XGBRegressor

import torch
import torch.nn as nn

# Global Variable

In [23]:
class GlobalVariable:
    def __init__(self, base_path='/kaggle/input/ariel-data-challenge-2025'):
        self.base_path = base_path
        self.adc_info = pd.read_csv(f'{base_path}/adc_info.csv')
        self.axis_info = pd.read_parquet(f'{base_path}/axis_info.parquet')
        self.train_labels = pd.read_csv(f'{base_path}/train.csv', index_col='planet_id')
        self.train_info = pd.read_csv(f'{base_path}/train_star_info.csv', index_col='planet_id')
        self.test_info = pd.read_csv(f'{base_path}/test_star_info.csv', index_col='planet_id')
        self.sample_test = pd.read_csv(f'{base_path}/sample_submission.csv', index_col='planet_id')
        self.dataset = 'train'
        self.cut_inf ,self.cut_sup = 39, 321
        self.nums_parallel = 4

        gainAIRS = self.adc_info['AIRS-CH0_adc_gain'].iloc[0]
        gainFGS1 = self.adc_info['FGS1_adc_gain'].iloc[0]
        offsetAIRS = self.adc_info['AIRS-CH0_adc_offset'].iloc[0]
        offsetFGS1 = self.adc_info['FGS1_adc_offset'].iloc[0]
        self.adc = {
            'AIRS-CH0': {'gain': gainAIRS, 'offset': offsetAIRS},
            'FGS1': {'gain': gainFGS1, 'offset': offsetFGS1}
        }

# Calibrating and binning data

In [24]:
# Tham khảo từ https://www.kaggle.com/code/gordonyip/calibrating-and-binning-ariel-data
class DataPreprocessing:
    def __init__(self, config):
        self.config = config

    # Sửa sai tuyến tính data
    def apply_linear_corr(self, linear_corr,clean_signal):
        linear_corr = np.flip(linear_corr, axis=0)
        for x, y in itertools.product(
                    range(clean_signal.shape[1]), range(clean_signal.shape[2])
                ):
            poli = np.poly1d(linear_corr[:, x, y])
            clean_signal[:, x, y] = poli(clean_signal[:, x, y])
        return clean_signal

    # trừ dark current
    def clean_dark(self, signal, dark, dt):
        dark = np.tile(dark, (signal.shape[0], 1, 1))
        signal -= dark* dt[:, np.newaxis, np.newaxis]
        return signal

    # tiền xử lý dữ liệu đầu vào và binning
    def preproc(self, dataset, sensor, binning = 15):
        
        sensor_sizes_dict = {"AIRS-CH0":[[11250, 32, 356], [1, 32, 356]], "FGS1":[[135000, 32, 32], [1, 32, 32]]}
        binned_dict = {"AIRS-CH0":[11250 // binning // 2, 356], "FGS1":[135000 // binning // 2]}
        linear_corr_dict = {"AIRS-CH0":(6, 32, 356), "FGS1":(6, 32, 32)}
        planet_ids = self.config.train_labels.index

        # phân biệt dang là dữ liệu train hay test
        if dataset == "test":
            sample_sub_path = f"{self.config.base_path}/sample_submission.csv"
            planet_ids = pd.read_csv(sample_sub_path)["planet_id"].tolist()
        
        feats = []

        # đọc qua từng hành tinh các thông số của nó và xử lý
        for i, planet_id in tqdm(list(enumerate(planet_ids))):
            signal = pd.read_parquet(f'{self.config.base_path}/{dataset}/{planet_id}/{sensor}_signal_0.parquet').to_numpy()
            dark_frame = pd.read_parquet(f'{self.config.base_path}/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration_0/dark.parquet', engine='pyarrow').to_numpy()
            dead_frame = pd.read_parquet(f'{self.config.base_path}/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration_0/dead.parquet', engine='pyarrow').to_numpy()
            flat_frame = pd.read_parquet(f'{self.config.base_path}/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration_0/flat.parquet', engine='pyarrow').to_numpy()
            linear_corr = pd.read_parquet(f'{self.config.base_path}/{dataset}/' + str(planet_id) + '/' + sensor + '_calibration_0/linear_corr.parquet').values.astype(np.float64).reshape(linear_corr_dict[sensor])

            # chuyển đổi tín hiệu về pixcel voltage
            signal = signal.reshape(sensor_sizes_dict[sensor][0]) 
            gain = self.config.adc[sensor]['gain']
            offset = self.config.adc[sensor]['offset']
            signal = signal / gain + offset

            # phát hiển pixcel hot
            hot = sigma_clip(
                dark_frame, sigma=5, maxiters=5
            ).mask

            # xử lý với các loại tín hiệu
            if sensor != "FGS1":
                dt = np.ones(len(signal))*0.1 
                dt[1::2] += 4.5
            else:
                dt = np.ones(len(signal))*0.1
                dt[1::2] += 0.1


            # gán 0 cho những phần từ nhỏ hơn 0
            signal = signal.clip(0)
            linear_corr_signal = self.apply_linear_corr(linear_corr, signal)
            signal = self.clean_dark(linear_corr_signal, dark_frame, dt)

            # chia dư liệu cho flat (độ lệch giữa các pixcel gốc)
            flat = flat_frame.reshape(sensor_sizes_dict[sensor][1])
            flat[dead_frame.reshape(sensor_sizes_dict[sensor][1])] = np.nan
            flat[hot.reshape(sensor_sizes_dict[sensor][1])] = np.nan
            signal = signal / flat
            
            # cắt ở trung tâm thu gọn dữ liệu lấy những dữ liệu chuẩn nhất phù hợp với bước sóng thứ nhất test
            if sensor == "FGS1":
                signal = signal[:,10:22,10:22] 
                signal = signal.reshape(sensor_sizes_dict[sensor][0][0],144) 

            # cắt ở trung tâm thu gọn dữ liệu lấy những dữ liệu chuẩn nhất
            if sensor != "FGS1":
                signal = signal[:,10:22,:]

            # trừ 2 pixcel chẵn lẻ
            mean_signal = np.nanmean(signal, axis=1) 
            cds_signal = (mean_signal[1::2] - mean_signal[0::2])

            # binning thu gọn chắt lọc dữ liệu
            binned = np.zeros((binned_dict[sensor]))
            for j in range(cds_signal.shape[0] // binning):
                binned[j] = cds_signal[j*binning:j*binning+binning].mean(axis=0) 
                       
            if sensor == "FGS1":
                binned = binned.reshape((binned.shape[0],1))
            
            feats.append(binned)
            
        return np.stack(feats)


# Data Processing

In [ ]:
# xử lý dữ liệu
# Tham khảo từ https://www.kaggle.com/code/vitalykudelya/neurips-non-ml-transit-curve-fitting?scriptVersionId=250312271
# và từ https://www.kaggle.com/code/jiazhuang/adc2024-fix-window-weights-finetuning/notebook
class DataProcessing:
    def __init__ (self, config):
        self.config = config
        self.data_pre = DataPreprocessing(config)

        # đọc lấy các file nếu đã xử lý trước đó
        if os.path.exists('/kaggle/input/data03/f_raw_train.npy'):
            self.f_raw_train = np.load('/kaggle/input/data03/f_raw_train.npy')
            print("Loading f done")
        else :
            self.f_raw_train = self.data_pre.preproc(f'{self.config.dataset}',"FGS1", 30*12)
            self.f_raw_train.shape
            np.save('/kaggle/working/f_raw_train.npy', self.f_raw_train)
        
        if os.path.exists('/kaggle/input/data03/a_raw_train.npy'):
            self.a_raw_train = np.load('/kaggle/input/data03/a_raw_train.npy')
            print("Loading a done")
        else :
            self.a_raw_train = self.data_pre.preproc(f'{self.config.dataset}',"AIRS-CH0", 30)
            self.a_raw_train.shape
            np.save('/kaggle/working/a_raw_train.npy', self.a_raw_train)

        # đọc dữ liệu cần test
        self.f_raw_test = self.data_pre.preproc('test',"FGS1", 30*12)
        self.a_raw_test = self.data_pre.preproc('test',"AIRS-CH0", 30)

    # làm mượt tín hiệu AIRS và cắt lấy những bước sóng ở giữa 
    # mượt bằng cách lấy chung bình 37 bước trái phải
    def smooth_airs(self,a_raw_data, window = 37):
        wv_smooth_a_raw = []
        for i in range(self.config.cut_inf, self.config.cut_sup):
           wv_smooth_a_raw.append(a_raw_data[:, :, max(i-window, 0):(i+window)].mean(axis=-1))
    
        wv_smooth_a_raw = np.stack(wv_smooth_a_raw, axis=-1)
        wv_smooth_a_raw.shape

        return wv_smooth_a_raw

    # function tìm ra điểm transit của tín hiệu gốc bằng cách tính đạo hàm của các điểm trên signal
    def phase_detector(self, signal):
        
        MIN = np.argmin(signal[30:140])+30
        signal1 = signal[:MIN ]
        signal2 = signal[MIN :]
    
        first_derivative1 = np.gradient(signal1)
        first_derivative1 /= first_derivative1.max()
        first_derivative2 = np.gradient(signal2)
        first_derivative2 /= first_derivative2.max()
    
        phase1 = np.argmin(first_derivative1)
        phase2 = np.argmax(first_derivative2) + MIN
    
        return phase1, phase2

    # function tìm giá trị lệch nhỏ nhất của tín hiệu cho vào sau khi * s ở transit với giá trị của một đa thức bậc nhỏ hơn 4 khớp nhất với tín hiệu
    # mục tiêu là minimal q tìm ra giá trị s phù hợp nhất
    def objective(self, signal, p1, p2, s):
        
        best_q = 1e10
        n = signal.shape[0]
        try:
            for i in range(4) :
                delta = 2
                end_before   = max(p1 - delta, 0) 
                start_after   = min(p2 + delta, n)
                y = signal[:end_before].tolist() + (signal[p1+delta:p2 - delta] * (1 + s)).tolist() + signal[start_after:].tolist()
                x = list(range(len(y)))
                z = np.polyfit(x, y, deg=i)
                p = np.poly1d(z)
                q = np.abs(p(x) - y).mean()
        
            if q < best_q :
                best_q = q
        
            return q
        except Exception as e:
            print(f"[ERROR objective()] {signal.shape} p1={p1}, p2={p2} x={len(x)}, y={len(y)}, p1={p1}, p2={p2}, s={s}, err={e}")
            return 1e10

    # tìm s bằng Nelder-Mead tự điều chỉnh sao cho q min
    def fit_transit_depth(self, signal, phase_signal):
        p1, p2 = self.phase_detector(phase_signal)
        f = partial(self.objective, signal, p1, p2)
        r = minimize(f, [0.0001], method= 'Nelder-Mead')
        s = r.x[0]
        return s

    # tìm giá trị s cho từng bước sóng
    def fit_every_wv(self, a_raw, phase_signal):
        res = []
        for i in range(282):
            signal = a_raw[:, i]
            res.append(
                self.fit_transit_depth(signal, phase_signal)
            )
        return np.array(res, dtype=np.float32).clip(0)

    # làm mượt dữ liệu bằng đa thức bậc 3 và nội suy trong khoảng +- windowsize/2
    def smooth_data(self, data, window_size):
        return savgol_filter(data, window_size, 3)

    # xử lý dữ liệu
    def process_data(self, a_raw_data, f_raw_data, type_data = 'train'):

        # tổng số dòng
        if type_data == 'train':
            total_rows = self.config.train_labels.shape[0]
        else:
            total_rows =len(self.config.sample_test.index)

        # làm mượt AIRS
        wv_smooth_a_raw = self.smooth_airs(a_raw_data)

        # cắt tín hiệu
        a_raw_mean = a_raw_data[:, :, self.config.cut_inf:self.config.cut_sup].mean(axis=-1)
        a_raw_mean.shape

        # tính toán một độ sâu duy nhất của FGS1 (wl dự đoán chỉ có duy nhất 1 bước sóng thuộc FGS1)
        with ThreadPoolExecutor(max_workers=self.config.nums_parallel) as exe:
            f_depth = list( tqdm(exe.map(self.fit_transit_depth, f_raw_data[:, :, 0], a_raw_mean), total=total_rows) )
        f_depth = np.array(f_depth, dtype=np.float32)

        # đọc xem đã có file độ sâu của 282 bước sóng chưa
        if os.path.exists('/kaggle/input/data03/a_wv_depth.npy') and type_data == 'train': 
            a_wv_depth = np.load('/kaggle/input/data03/a_wv_depth.npy')
        else :
            a_wv_depth = Parallel(
                n_jobs=self.config.nums_parallel,
                backend='loky',
                prefer='processes'
            )(
                delayed(self.fit_every_wv)(ws, m)
                for ws, m in tqdm(zip(wv_smooth_a_raw, a_raw_mean),
                                   total=total_rows)
            )
            a_wv_depth = np.vstack(a_wv_depth)
            if type_data == 'train':
                np.save('/kaggle/working/a_wv_depth.npy', a_wv_depth)


        # làm mượt độ sâu với hàm bậc 3 và nội suy
        window_size = 103
        smoothed = []
        for pred in tqdm(a_wv_depth):
            smooth_pred = self.smooth_data(pred, window_size=window_size)
            smoothed.append(smooth_pred)
        smoothed = np.row_stack(smoothed)
        smoothed.shape
            
        a_wv_depth = smoothed

        window = 64
        tail_a_raw = a_raw_data[:,:, -window:].mean(axis=-1)
        with ThreadPoolExecutor(max_workers=self.config.nums_parallel) as exe:
            tail_a_depth = list( tqdm(exe.map(self.fit_transit_depth, tail_a_raw, a_raw_mean), total=total_rows) )

        tail_a_depth = np.array(tail_a_depth, dtype=np.float32)
        merged_f_depth = tail_a_depth * 0.9 + f_depth * 0.1


        # chuẩn hóa dữ liệu meta data của các hành tinh
        scaler_meta = StandardScaler()
            
        if type_data == 'train':
            meta_features = scaler_meta.fit_transform(self.config.train_info.values)
        else:
            meta_features = scaler_meta.fit_transform(self.config.test_info.values)

        wv_smooth_pred = np.column_stack([
            merged_f_depth,
            a_wv_depth[:, ::-1]
        ])
        wv_smooth_pred.shape

        # ghép dữ liệu meta data với dữ liệu đô sâu s
        X = np.hstack([wv_smooth_pred, meta_features])
        return X


# Develop Model

In [26]:
# Xây dựng model
class DevelopedModel:
    def __init__(self,
                 mu_model_path="/kaggle/input/model_001/scikitlearn/default/3/mu_model.pkl",
                 sigma_model_path="/kaggle/input/model_001/scikitlearn/default/3/sigma_model.pkl"):

        self.mu_model_path = mu_model_path
        self.sigma_model_path = sigma_model_path
        self.mu_model = None
        self.sigma_model = None
        self.trained = False

        if os.path.exists(self.mu_model_path) and os.path.exists(self.sigma_model_path):
            print("Loading existing models...")
            self.mu_model = joblib.load(self.mu_model_path)
            self.sigma_model = joblib.load(self.sigma_model_path)

    # traning model với tập dữ liệu X và đầu ra y với test mặc định là 10%
    def fit(self, X, y, train_size=0):
        if self.mu_model is not None and self.sigma_model is not None:
            print("Models already loaded. Skipping training.")
            return

        # self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
        #     X, y, train_size=train_size, random_state=42
        # )

        self.X_train = X
        self.y_train = y
        self.X_test = None
        self.y_test = None

        base_model_sigma = XGBRegressor(
            n_estimators=300,
            max_depth=6,
            learning_rate=0.05,
            n_jobs=-1,
            random_state=42
        )

        base_model = LinearRegression()
        eps = 1e-6


        print("Developing model ...")

        self.mu_model = MultiOutputRegressor(base_model)
        mu_oof = cross_val_predict(self.mu_model, self.X_train, self.y_train, cv=5)
        sigma_true = np.abs(self.y_train - mu_oof)

        self.sigma_model = MultiOutputRegressor(base_model_sigma)
        
        self.mu_model.fit(self.X_train, self.y_train)
        self.sigma_model.fit(self.X_train, sigma_true)
        #     base_model = SVR(
        #         kernel='rbf',
        #         C=1.0,
        #         epsilon=0.1
        #     )

        # base_model = Ridge(alpha=1.0)

        # base_model = Lasso(alpha=0.1, max_iter=10000)

        # Model xử dụng
        # base_model = LinearRegression()
        
        print("Saving models to /kaggle/working/ ...")
        joblib.dump(self.mu_model, "/kaggle/working/mu_model.pkl")
        joblib.dump(self.sigma_model, "/kaggle/working/sigma_model.pkl")

        self.trained = True

    def predict(self, X):
        if self.mu_model is None or self.sigma_model is None:
            raise RuntimeError("Model not trained or loaded.")
        mu_pred = self.mu_model.predict(X)
        sigma_pred = self.sigma_model.predict(X)
        return mu_pred, sigma_pred

    def evaluate(self):
        if not self.trained:
            print("Model was loaded (not trained), skipping evaluation plots.")
            return
    
        mu_pred, sigma_pred = self.predict(self.X_test)
        abs_errors = np.abs(self.y_test - mu_pred)
        mse = mean_squared_error(self.y_test, mu_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(self.y_test, mu_pred)
        r2 = r2_score(self.y_test, mu_pred)
        mean_sigma = np.mean(sigma_pred)
     
        print(f"MAE: {mae:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print(f"R² Score: {r2:.4f}")
        print(f"Mean predicted sigma (uncertainty): {mean_sigma:.4f}")
    
        # Vẽ biểu đồ
        plt.figure(figsize=(18, 5))
    
        plt.subplot(1, 3, 1)
        plt.scatter(self.y_test.ravel(), mu_pred.ravel(), alpha=0.3)
        plt.xlabel("True")
        plt.ylabel("Predicted")
        plt.title("True vs Predicted")
    
        plt.subplot(1, 3, 2)
        plt.hist(abs_errors.ravel(), bins=50, color='orange', alpha=0.8)
        plt.xlabel("Absolute Error")
        plt.title("Distribution of Absolute Errors")
        
        plt.tight_layout()
        plt.show()

# Create Submission

In [27]:
# Tạo submission in ra file
class SubmissionGenerator:
    def __init__(self, model):
        self.model = model

    def predict(self, X_new):
        mu, sigma = self.model.predict(X_new)
        return mu, sigma

    def save_submission(self, mu, sigma=None, filename="submission.csv", sample_path="/kaggle/input/ariel-data-challenge-2025/sample_submission.csv"):
        planet_ids = pd.read_csv(sample_path)["planet_id"].astype(int).tolist()
        mu_columns = [f"wl_{i+1}" for i in range(283)]
        sigma_columns = [f"sigma_{i+1}" for i in range(283)]

        submission_df = pd.DataFrame(
        np.hstack([mu, sigma]),
            columns=mu_columns + sigma_columns
        )
        submission_df.insert(0, "planet_id", planet_ids)
        submission_df["planet_id"] = submission_df["planet_id"].astype(int)
        submission_df.to_csv(filename, index=False, float_format="%.6f")
        print(f"Submission saved to {filename}")

# Run

In [ ]:
# Load và xử lý dữ liệu
config = GlobalVariable()
data_processing = DataProcessing(config)

a_raw_train = data_processing.a_raw_train
f_raw_train = data_processing.f_raw_train

X_train = data_processing.process_data(a_raw_train,f_raw_train)
y_mu  = config.train_labels.values

Loading f done
Loading a done


 37%|███▋      | 405/1100 [00:35<00:59, 11.68it/s]

In [ ]:
# Huấn luyện và lưu model
model = DevelopedModel()
model.fit(X_train, y_mu)
# model.evaluate()

In [ ]:
# Load và xử lý dữ liệu test
a_raw_test = data_processing.a_raw_test
f_raw_test = data_processing.f_raw_test

X_test = data_processing.process_data(a_raw_test,f_raw_test,'test')

In [ ]:
# Load model và tạo file submission
submission_gen = SubmissionGenerator(model)
mu_pred, sigma_pred = submission_gen.predict(X_test)
submission_gen.save_submission(mu_pred, sigma_pred)